# BNSynthesis-generator

## Instructions

1. Set `N` and `M` to the desired values

1. Run the script
   - Just run the cell
   - This will generate a python script with fully underspecified original BN `(X,F)`, abstraction function `G`, and abstract BN `(Y,H)`

1. Refine any of the above at will (look for comment "# Replace rhs with actual constraints")
   - For example, to synthetize an abstraction function for a given original BN, one can specify the original BN by specifying each update function `fi` like this
> `ForAll([x0,x1,..,xN],fi(x0,x1,..,xN) == ...`
   -The LHS is ready in the generated file. One has just to replace the RHS.

1. Run the new script as usual
   - This will generate possible ways of completing `F`, `G` and `H`


## Script generator

In [1]:
from IPython.display import display, Markdown#, Latex
display(Markdown('__Generated on 2022-11-30 08:38:58.349721__'))
display(Markdown("N=3, M=2"))


# Install z3 for python as explained here
# https://github.com/Z3Prover/z3
from z3 import *

# The solver
solver = Solver()

# Original BN update function F = f0..fn-1
f0 = Function('f0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f1 = Function('f1', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f2 = Function('f2', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables for original BN
x0 = Const('x0' , BoolSort())
x1 = Const('x1' , BoolSort())
x2 = Const('x2' , BoolSort())

# Abstraction function G = g0..gn-1
g0 = Function('g0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
g1 = Function('g1', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Abstract BN update function H = h1..hn-1
h0 = Function('h0', BoolSort(),BoolSort(),BoolSort())
h1 = Function('h1', BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables
y0 = Const('y0' , BoolSort())
y1 = Const('y1' , BoolSort())

# Constraints (partially) defining F, G and H
solver.add(
    # Vacuous constraints for f to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],f0(x0,x1,x2) == f0(x0,x1,x2)),
    ForAll([x0,x1,x2],f1(x0,x1,x2) == f1(x0,x1,x2)),
    ForAll([x0,x1,x2],f2(x0,x1,x2) == f2(x0,x1,x2)),

    # Vacuous constraints for g to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],g0(x0,x1,x2) == g0(x0,x1,x2)),
    ForAll([x0,x1,x2],g1(x0,x1,x2) == g1(x0,x1,x2)),

    # Vacuous constraints for h to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1],h0(x0,x1) == h0(x0,x1)),
    ForAll([x0,x1],h1(x0,x1) == h1(x0,x1)),

    #Determinism condition (i.e. existence of abstract bisimilar deterministic/synchronous BN)
    ForAll([x0,x1,x2], And(
      g0(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h0(g0(x0,x1,x2),g1(x0,x1,x2)),
      g1(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h1(g0(x0,x1,x2),g1(x0,x1,x2))
    ))
)

#While loop providing solutions
while solver.check() == sat:
    m = solver.model()
    print("Hoorray! Here is a possible solution:")
    print(m)
    print("Press [enter] for a different one, or type stop to terminate...")
    string = input()
    if string == 'STOP' or string == 'stop':
        print('I terminate, as required')
        break
    solver.add(Or([
      # Constraints to obtain a different solution (for now only a different G)
      Not(ForAll([x0,x1,x2],m.eval(g0(x0,x1,x2)) == g0(x0,x1,x2))),
      Not(ForAll([x0,x1,x2],m.eval(g1(x0,x1,x2)) == g1(x0,x1,x2))),
      False]
    ))
else:
    print("No solution, sorry!")


Generation completed. Creating new cell with computed output...
done


# Precomputed scripts

## Original generated script

In [ ]:
#N = 3, M = 2

# Install z3 for python as explained here
# https://github.com/Z3Prover/z3
from z3 import *

# The solver
solver = Solver()

# Original BN update function F = f0..fn-1
f0 = Function('f0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f1 = Function('f1', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f2 = Function('f2', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables for original BN
x0 = Const('x0' , BoolSort())
x1 = Const('x1' , BoolSort())
x2 = Const('x2' , BoolSort())

# Abstraction function G = g0..gn-1
g0 = Function('g0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
g1 = Function('g1', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Abstract BN update function H = h1..hn-1
h0 = Function('h0', BoolSort(),BoolSort(),BoolSort())
h1 = Function('h1', BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables
y0 = Const('y0' , BoolSort())
y1 = Const('y1' , BoolSort())

# Constraints (partially) defining F, G and H
solver.add(
    # Vacuous constraints for f to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],f0(x0,x1,x2) == f0(x0,x1,x2)),
    ForAll([x0,x1,x2],f1(x0,x1,x2) == f1(x0,x1,x2)),
    ForAll([x0,x1,x2],f2(x0,x1,x2) == f2(x0,x1,x2)),

    # Vacuous constraints for g to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],g0(x0,x1,x2) == g0(x0,x1,x2)),
    ForAll([x0,x1,x2],g1(x0,x1,x2) == g1(x0,x1,x2)),

    # Vacuous constraints for h to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1],h0(x0,x1) == h0(x0,x1)),
    ForAll([x0,x1],h1(x0,x1) == h1(x0,x1)),

    #Determinism condition (i.e. existence of abstract bisimilar deterministic/synchronous BN)
    ForAll([x0,x1,x2], And(
      g0(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h0(g0(x0,x1,x2),g1(x0,x1,x2)),
      g1(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h1(g0(x0,x1,x2),g1(x0,x1,x2))
    ))
)

#While loop providing solutions
while solver.check() == sat:
    m = solver.model()
    print("Hoorray! Here is a possible solution:")
    print(m)
    print("Press [enter] for a different one, or type stop to terminate...")
    string = input()
    if string == 'STOP' or string == 'stop':
        print('I terminate, as required')
        break
    solver.add(Or([
      # Constraints to obtain a different solution (for now only a different G)
      Not(ForAll([x0,x1,x2],m.eval(g0(x0,x1,x2)) == g0(x0,x1,x2))),
      Not(ForAll([x0,x1,x2],m.eval(g1(x0,x1,x2)) == g1(x0,x1,x2))),
      False]))
else:
    print("No solution, sorry!")

Hoorray! Here is a possible solution:
[g0 = [else -> False],
 f0 = [else -> False],
 f2 = [else -> False],
 f1 = [else -> False],
 h0 = [else -> False],
 h1 = [else -> False],
 g1 = [else -> False]]
Press [enter] for a different one, or type stop to terminate...


 stop


I terminate, as required


## Refined generated script

In [2]:
#N = 3, M = 2

# Install z3 for python as explained here
# https://github.com/Z3Prover/z3
from z3 import *

# The solver
solver = Solver()

# Original BN update function F = f0..fn-1
f0 = Function('f0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f1 = Function('f1', BoolSort(),BoolSort(),BoolSort(),BoolSort())
f2 = Function('f2', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables for original BN
x0 = Const('x0' , BoolSort())
x1 = Const('x1' , BoolSort())
x2 = Const('x2' , BoolSort())

# Abstraction function G = g0..gn-1
g0 = Function('g0', BoolSort(),BoolSort(),BoolSort(),BoolSort())
g1 = Function('g1', BoolSort(),BoolSort(),BoolSort(),BoolSort())

# Abstract BN update function H = h1..hn-1
h0 = Function('h0', BoolSort(),BoolSort(),BoolSort())
h1 = Function('h1', BoolSort(),BoolSort(),BoolSort())

# Some Boolean variables
y0 = Const('y0' , BoolSort())
y1 = Const('y1' , BoolSort())

# Constraints (partially) defining F, G and H
solver.add(
    # Vacuous constraints for f to serve as placehoders
    # Replace rhs with actual constraints
    #ForAll([x0,x1,x2],f0(x0,x1,x2) == f0(x0,x1,x2)),
    #ForAll([x0,x1,x2],f1(x0,x1,x2) == f1(x0,x1,x2)),
    #ForAll([x0,x1,x2],f2(x0,x1,x2) == f2(x0,x1,x2)),
    # BN from Georgios slides
    ForAll([x0,x1,x2],f0(x0,x1,x2) == Not(x1)) ,
    ForAll([x0,x1,x2],f1(x0,x1,x2) == Or(Not(x0),Not(x2))) ,
    ForAll([x0,x1,x2],f2(x0,x1,x2) == Or([Not(x0),x1,x2])) ,

    # Vacuous constraints for g to serve as placehoders
    # Replace rhs with actual constraints
    ForAll([x0,x1,x2],g0(x0,x1,x2) == g0(x0,x1,x2)),
    ForAll([x0,x1,x2],g1(x0,x1,x2) == g1(x0,x1,x2)),

    # Vacuous constraints for h to serve as placehoders
    # Replace rhs with actual constraints
    #ForAll([x0,x1],h0(x0,x1) == h0(x0,x1)),
    #ForAll([x0,x1],h1(x0,x1) == h1(x0,x1)),
    # Aggregation function slide 2 - Does not provide a bisimilar deterministic/synchronous BN
    #ForAll([x0,x1,x2],g0(x0,x1,x2) == Or(x0,x1)) ,
    #ForAll([x0,x1,x2],g1(x0,x1,x2) == x2) ,
    # Aggregation function slide 6 - Provides a unique bisimilar deterministic/synchronous BN
    ForAll([x0,x1,x2],g0(x0,x1,x2) == Or(Not(x0),x1)) ,
    ForAll([x0,x1,x2],g1(x0,x1,x2) == x2) ,

    #Determinism condition (i.e. existence of abstract bisimilar deterministic/synchronous BN)
    ForAll([x0,x1,x2], And(
      g0(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h0(g0(x0,x1,x2),g1(x0,x1,x2)),
      g1(f0(x0,x1,x2),f1(x0,x1,x2),f2(x0,x1,x2)) == h1(g0(x0,x1,x2),g1(x0,x1,x2))
    ))
)

#While loop providing solutions
while solver.check() == sat:
    m = solver.model()
    print("Hoorray! Here is a possible solution:")
    print(m)
    print("Press [enter] for a different one, or type stop to terminate...")
    string = input()
    if string == 'STOP' or string == 'stop':
        print('I terminate, as required')
        break
    solver.add(Or([
      # Constraints to obtain a different solution (for now only a different G)
      Not(ForAll([x0,x1,x2],m.eval(g0(x0,x1,x2)) == g0(x0,x1,x2))),
      Not(ForAll([x0,x1,x2],m.eval(g1(x0,x1,x2)) == g1(x0,x1,x2))),
      False]))
else:
    print("No solution, sorry!")

Hoorray! Here is a possible solution:
[f0 = [else ->
       Or(And(Not(Var(0)), Not(Var(1)), Var(2)),
          And(Var(0), Not(Var(1)), Var(2)),
          And(Var(0), Not(Var(1)), Not(Var(2))),
          And(Not(Var(0)), Not(Var(1)), Not(Var(2))))],
 g0 = [else ->
       Or(And(Not(Var(0)), Not(Var(1)), Var(2)),
          And(Not(Var(0)), Not(Var(1)), Not(Var(2))),
          And(Not(Var(0)), Var(1), Var(2)),
          And(Var(0), Var(1), Not(Var(2))),
          And(Not(Var(0)), Var(1), Not(Var(2))),
          And(Var(0), Var(1), Var(2)))],
 h1 = [(False, False) -> False, else -> True],
 f2 = [else ->
       Or(And(Not(Var(0)), Not(Var(1)), Var(2)),
          And(Var(0), Not(Var(1)), Var(2)),
          And(Not(Var(0)), Not(Var(1)), Not(Var(2))),
          And(Not(Var(0)), Var(1), Var(2)),
          And(Var(0), Var(1), Not(Var(2))),
          And(Not(Var(0)), Var(1), Not(Var(2))),
          And(Var(0), Var(1), Var(2)))],
 h0 = [(False, True) -> False, else -> True],
 f1 = [else ->
     

 stop


I terminate, as required
